In [17]:
import os
import numpy as np 
import pandas as pd
from scipy.io import loadmat
from keras.utils.np_utils import to_categorical

In [18]:
path_to_train = '/media/sf_VBox_Shared/timeseries/KaggleEEG/train_1/'

In [19]:
def mat_to_data(path, max_timesteps=None):
    try :
        mat = loadmat(path)
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        if max_timesteps is None:
            return ndata['data']
        else:
            return ndata['data'][:max_timesteps,:]
    except :
        return None

In [20]:
fnames = os.listdir(path_to_train)

In [21]:
fnames_split = [fname.split('_') for fname in fnames if fname[-4:]=='.mat']
metadata = pd.DataFrame([{'patient':int(f[0]), 'segment':int(f[1]), 'class':int(f[2][0])} for f in fnames_split])

In [22]:
nr_val = 10*6

In [23]:
metadata_0 = metadata[metadata['class']==0]
metadata_1 = metadata[metadata['class']==1]

nr_segments_0 = metadata_0['segment']

In [24]:
np.random.seed(123)
ind_0 = metadata_0.sort_values('segment').index
ind_1 = metadata_1.sort_values('segment').index
val_ind = np.random.permutation(ind_0[-nr_val:].append( ind_1[-nr_val:]))
train_ind = np.random.permutation(ind_0[:-nr_val].append(ind_1[:-nr_val]))[:100]

In [25]:
def read_data(filenames, path_to_train, max_timesteps=None):
    mfiles = [mat_to_data(os.path.join(path_to_train, fname), max_timesteps) for fname in filenames]
    data = [m for m in mfiles if m is not None]
    return np.array(data)

In [26]:
X_val = read_data(pd.Series(fnames)[val_ind], path_to_train)
y_val = metadata.loc[val_ind]['class']
y_val_binary = to_categorical(np.array(y_val), 2)

In [27]:
outdatapath = '/media/sf_VBox_Shared/timeseries/KaggleEEG/train_1/prepared/'
np.save(outdatapath+'X_val', X_val)
np.save(outdatapath+'y_val_binary', y_val_binary)

In [32]:
X_train = read_data(pd.Series(fnames)[train_ind], path_to_train, max_timesteps=2400)
y_train = metadata.loc[train_ind]['class']
y_train_binary = to_categorical(np.array(y_train), 2)
np.save(outdatapath+'X_train', X_train)
np.save(outdatapath+'y_train_binary', y_train_binary)

In [29]:
X_train.shape

(100, 24000, 16)

In [30]:
np.save(outdatapath+'y_val_binary', y_val_binary)
np.save(outdatapath+'y_train_binary', y_train_binary)